In [6]:
"""국토교통부 아파트전월세 실거래 데이터 추출 예시
- 링크: https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058017
"""
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm
import xmltodict


# load .env
load_dotenv()
getRTMSDataSvcAptTradeDev_en = os.environ.get("getRTMSDataSvcAptTradeDev_en")
getRTMSDataSvcAptTradeDev_de = os.environ.get("getRTMSDataSvcAptTradeDev_de")

In [21]:
start_date = pd.to_datetime('1980-01-01')
end_date = pd.to_datetime('2022-10-31')
dates = pd.date_range(start_date, end_date, freq="M")

In [22]:
MONTH_LIST = [date.strftime("%Y%m") for date in dates]
SIGUNGU_CODE = 11470

In [23]:
url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent"

In [24]:
df_all = pd.DataFrame()

for month in tqdm(MONTH_LIST):
    params = {
        "serviceKey": getRTMSDataSvcAptTradeDev_de,
        "LAWD_CD": SIGUNGU_CODE,  # Input 1: 시군구코드 5자리, ex) 11680 = 서울특별시 강남구
        "DEAL_YMD": month,  # Input 2: 년월(YYYYMM), ex) 202208 = 2022년 8월
    }

    response = requests.get(url, params=params)

    if xmltodict.parse(response.content)["response"]["body"]["items"]:
        dict_content = xmltodict.parse(response.content)["response"]["body"]["items"]["item"]
        df = pd.DataFrame(dict_content)
        df_all = pd.concat([df_all, df])

    # df.to_csv(f"data/서울특별시_양천구_아파트전월세/서울특별시_양천구_아파트전월세_{month}.csv",
    #           encoding="utf8",
    #           index=False
    # )

100%|██████████| 514/514 [02:41<00:00,  3.19it/s]


In [25]:
df_all.to_csv("data/서울특별시_양천구_아파트전월세_1980_202210.csv", index=False)

In [26]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105285 entries, 0 to 767
Data columns (total 17 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   갱신요구권사용  3689 non-null    object
 1   건축년도     105285 non-null  object
 2   계약구분     11553 non-null   object
 3   계약기간     12223 non-null   object
 4   년        105285 non-null  object
 5   법정동      105285 non-null  object
 6   보증금액     105285 non-null  object
 7   아파트      105285 non-null  object
 8   월        105285 non-null  object
 9   월세금액     105285 non-null  object
 10  일        105285 non-null  object
 11  전용면적     105285 non-null  object
 12  종전계약보증금  9468 non-null    object
 13  종전계약월세   9468 non-null    object
 14  지번       105285 non-null  object
 15  지역코드     105285 non-null  object
 16  층        105285 non-null  object
dtypes: object(17)
memory usage: 14.5+ MB
